In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#lyratas
edall_accessions = [ "SiberianAly_8chr.fasta.csv", "MN47Hifi_scafNT1.fasta.csv"]


lengths = [[175,180],[165,170]]#AlCEN178 an AlCEN168

edallf = []

for i in edall_accessions: 
    #load in repeats for accession
    path = r"C:\Users\Christie and Victori\Documents\christie\work uni\part II\project\cleaning_sequences\drive-download-20211014T102922Z-001\ed25.11.21.all.repeats.from."+i
    accession_df = pd.read_csv(path)
    
    filelist = accession_df["fasta.file.name"].tolist()
    newfile = []
    for f in filelist:
        new = f.replace(".fasta","")
        newfile.append(new)
    accession_df["fasta.file.name"] = newfile

    chromlist = accession_df["chromosome"].tolist()
    newchrom = []
    replace = ["Aarenosa_", "scaffold", "RaGOO", "CM032905", "_RagTag", "_", "Asue"]
    for k in chromlist:
        elem = k.replace("chr","Chr")
        for r in replace:
            elem1 = elem.replace(r, "")
            elem = elem1
        if len(elem) < 3:
            elem = "Chr" + elem
        newchrom.append(elem)
    accession_df["chromosome"] = newchrom

    accession_df["names"] = accession_df["fasta.file.name"].str.cat(accession_df["chromosome"].astype("str"), sep ="_") 
    accession_df["names"] = accession_df["names"].str.cat(accession_df["start"].astype("str"), sep ="_") 
    accession_df["names"] = accession_df["names"].str.cat(accession_df["width"].astype("str"), sep ="_") 
    # concatenate strings in fasta.name and start column to make names file; make sure variables are strings
    
    wanted = []
    for x in lengths:
        #subsetting repeats into cen180 and cen160
        cen180 = accession_df[accession_df["width"].between(x[0], x[1])]

        # # stratified sampling of repeats

        freqrep = cen180["chromosome"].value_counts()
        freqdf = pd.DataFrame(freqrep)
        length = cen180["chromosome"].count()
        total = 25
        freqdf["freq want"] = (total*freqrep.div(length)).astype(int)

        for x in range(len(freqdf)):
            y = cen180.get(cen180["chromosome"]== freqdf.index[x]).sample(n = freqdf.iloc[x,1])
            wanted.append(y)

    edallf = edallf + wanted #list of all rows in accessions

edall_df = pd.concat(edallf) #whole df!
lyratas = edall_df

In [3]:
#thalianas - cen180 and cen160
edall_accessions = ["t2t-col.20210610.fasta.csv"
                    ]

#AtCEN160 and AtCEN178
lengths = [[155,160],[175,180]]

edallf = []

for i in edall_accessions: 
    #load in repeats for accession
    path = r"C:\Users\Christie and Victori\Documents\christie\work uni\part II\project\cleaning_sequences\drive-download-20211014T102922Z-001\ed25.11.21.all.repeats.from."+i
    accession_df = pd.read_csv(path)
    
    filelist = accession_df["fasta.file.name"].tolist()
    newfile = []
    for f in filelist:
        new = f.replace(".fasta","")
        newfile.append(new)
    accession_df["fasta.file.name"] = newfile

    chromlist = accession_df["chromosome"].tolist()
    newchrom = []
    replace = ["Aarenosa_", "scaffold", "RaGOO", "CM032905", "_RagTag", "_", "Asue"]
    for k in chromlist:
        elem = k.replace("chr","Chr")
        for r in replace:
            elem1 = elem.replace(r, "")
            elem = elem1
        if len(elem) < 3:
            elem = "Chr" + elem
        newchrom.append(elem)
    accession_df["chromosome"] = newchrom

    accession_df["names"] = accession_df["fasta.file.name"].str.cat(accession_df["chromosome"].astype("str"), sep ="_") 
    accession_df["names"] = accession_df["names"].str.cat(accession_df["start"].astype("str"), sep ="_") 
    accession_df["names"] = accession_df["names"].str.cat(accession_df["width"].astype("str"), sep ="_") 
    # concatenate strings in fasta.name and start column to make names file; make sure variables are strings
    
    wanted = []
    for x in lengths:
        #subsetting repeats into cen180 and cen160
        cen180 = accession_df[accession_df["width"].between(x[0], x[1])]

        # # stratified sampling of repeats

        freqrep = cen180["chromosome"].value_counts()
        freqdf = pd.DataFrame(freqrep)
        length = cen180["chromosome"].count()
        total = 450
        freqdf["freq want"] = (total*freqrep.div(length)).astype(int)

        for x in range(len(freqdf)):
            y = cen180.get(cen180["chromosome"]== freqdf.index[x]).sample(n = freqdf.iloc[x,1])
            wanted.append(y)

    edallf = edallf + wanted #list of all rows in accessions

edall_df = pd.concat(edallf) #whole df!
thalianas = edall_df

In [4]:
merge = [thalianas, lyratas]
edsampled = pd.concat(merge) #merge dataframes
edsampled.rename(columns={"width":"length","seq":"sequence","chromosome":"fasta.name"}, inplace = True)
edsampled["region.start"] = edsampled["start"]
edsampled["region.end"] = edsampled["end"]
edsampled = edsampled[["fasta.name","start","end","length","sequence","strand","region.start","region.end","names"]]

sampled=edsampled
uniqsampled1 = sampled.drop_duplicates(subset = ["sequence"])
uniqsampled = uniqsampled1.drop_duplicates(subset = ["names"])

In [5]:
#make uniq sampled fasta

uniq_fasta = open(r"t2t_all_vlyrata_labelled.Fasta","w+") #open fasta file, "w+" means you intend to write in it

empty_uniq_fasta = [] #empty list to compile all rows
for j in range(len(uniqsampled)): #do for all rows
    string_uniq =uniqsampled.iloc[j,4]
    n=60
    split_uniq = [string_uniq[i:i+n] for i in range(0, len(string_uniq), n)] #splits sequence string into length 60
    join_uniq = "\n".join(split_uniq) #joins string with new lines
    write_fasta_uniq = ">"+uniqsampled.iloc[j,8]+"\n"+join_uniq #create fasta format
    empty_uniq_fasta.append(write_fasta_uniq) #append row's fasta to list
joined_fasta_uniq = "\n".join(empty_uniq_fasta) #join strings in list into one big string with new line breaks
uniq_fasta.write(joined_fasta_uniq) #enter list into fasta file
uniq_fasta.close() #close fasta file 